In [4]:
import numpy as np
import math
import pandas as pd

np.seterr(all="raise")  # Error on overflow

# INPUTS
LOAD_DEAD = 400  # kN
LOAD_LIVE = 200  # kN
BEARINGPRESSURE_ALLOWABLE = 250  # kPa
FTG_COVER = 0.075  # m
COLUMN_WIDTH = 0.400  # m

# Constants
REO_DENSITY = 7850  # kg/m3


class FoundationSizes:
    def __init__(self):
        """Initialize the FoundationSizes class and calculate all properties."""
        self.sizes = self.generate_foundation_sizes()
        self.initialize_attributes()
        self.calculate_all_properties()

    def generate_foundation_sizes(self):
        """Generate the foundation sizes based on initial parameters."""
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt((LOAD_DEAD + LOAD_LIVE) / BEARINGPRESSURE_ALLOWABLE) / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * COLUMN_WIDTH**2) / 3
                        + (1 + (3 * BEARINGPRESSURE_ALLOWABLE) / 19040)
                        * (LOAD_DEAD + LOAD_LIVE)
                    )
                    - (9520 + 3 * BEARINGPRESSURE_ALLOWABLE) * COLUMN_WIDTH
                )
                / (19040 + 3 * BEARINGPRESSURE_ALLOWABLE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        # Instantiate the full array with additional empty columns
        sizes = np.zeros((foundation_sizes.shape[0], 33), dtype=np.float32)
        sizes[:, :5] = foundation_sizes
        return sizes

    def initialize_attributes(self):
        """Initialize attributes for foundation properties."""
        attributes = [
            "FtgLength",
            "FtgDepth",
            "fc",
            "ReoSize",
            "ReoCts",
            "BPmax",
            "BP_Ratio",
            "Pult",
            "Dom",
            "BPult",
            "CLR",
            "VPult",
            "fVP",
            "VP_Ratio",
            "dv",
            "VLult",
            "kvo",
            "kv",
            "ks",
            "fVuc",
            "VL_Ratio",
            "Mult",
            "Astshr",
            "Astmin",
            "Ast",
            "alpha",
            "gamma",
            "ku",
            "phi",
            "Astact",
            "fMuo",
            "M_Ratio",
            "cost",
        ]

        for i, attr in enumerate(attributes):
            setattr(self, attr, self.sizes[:, i])

    def calculate_all_properties(self):
        """Calculate all foundation properties."""
        calc_methods = [
            self.calc_BPmax,
            self.calc_Pult,
            self.calc_ratio_bearing,
            self.calc_Dom,
            self.calc_BPult,
            self.calc_CLR,
            self.calc_VPult,
            self.calc_fVP,
            self.calc_VP_Ratio,
            self.calc_dv,
            self.calc_VLult,
            self.calc_kvo,
            self.calc_kv,
            self.calc_ks,
            self.calc_fVuc,
            self.calc_VL_Ratio,
            self.calc_Mult,
            self.calc_Astshr,
            self.calc_Astmin,
            self.calc_Ast,
            self.calc_alpha,
            self.calc_gamma,
            self.calc_ku,
            self.calc_phi,
            self.calc_Astact,
            self.calc_fMuo,
            self.calc_M_Ratio,
            self.calc_cost,
        ]

        for method in calc_methods:
            method()

    def calc_BPmax(self):
        """Calculate maximum bearing pressure."""
        self.BPmax[:] = (
            6 * self.FtgDepth * self.FtgLength**2 + LOAD_LIVE + LOAD_DEAD
        ) / (self.FtgLength**2)

    def calc_ratio_bearing(self):
        """Calculate bearing pressure ratio."""
        self.BP_Ratio[:] = self.BPmax / BEARINGPRESSURE_ALLOWABLE

    def calc_Pult(self):
        """Calculate ultimate load."""
        self.Pult[:] = (
            1.2 * (6 * self.FtgDepth * self.FtgLength**2 + LOAD_DEAD) + 1.5 * LOAD_LIVE
        )

    def calc_Dom(self):
        """Calculate depth of the moment arm."""
        self.Dom[:] = self.FtgDepth - FTG_COVER - self.ReoSize / 2

    def calc_BPult(self):
        """Calculate ultimate bearing pressure."""
        self.BPult[:] = self.Pult / self.FtgLength**2

    def calc_CLR(self):
        """Calculate column load ratio."""
        self.CLR[:] = self.BPult * (COLUMN_WIDTH + self.Dom) ** 2

    def calc_VPult(self):
        """Calculate ultimate vertical load."""
        self.VPult[:] = self.Pult - self.CLR

    def calc_fVP(self):
        """Calculate vertical load capacity"""
        self.fVP[:] = (
            952
            * (self.ReoSize - self.Dom)
            * (self.ReoSize - self.Dom - COLUMN_WIDTH)
            * np.sqrt(self.fc)
        )

    def calc_VP_Ratio(self):
        """Calculate vertical load capacity ratio"""
        self.VP_Ratio[:] = self.VPult / self.fVP

    def calc_dv(self):
        """Calculate vertical load displacement"""
        self.dv[:] = np.maximum(0.9 * self.Dom, 0.72 * self.FtgDepth)

    def calc_VLult(self):
        """Calculate ultimate vertical load"""
        self.VLult[:] = (
            -0.5 * self.BPult * (COLUMN_WIDTH + 2 * self.dv - self.FtgLength)
        )

    def calc_kvo(self):
        self.kvo[:] = 2 / (10 + 13 * self.dv)

    def calc_kv(self):
        self.kv[:] = np.minimum(self.kvo, 0.15)

    def calc_ks(self):
        self.ks[:] = np.maximum(0.5, (10 / 7) * (1 - self.FtgDepth))

    def calc_fVuc(self):
        self.fVuc[:] = 700 * self.dv * np.sqrt(self.fc) * self.ks * self.kv

    def calc_VL_Ratio(self):
        self.VL_Ratio[:] = self.VLult / self.fVuc

    # Mult per metre
    def calc_Mult(self):
        self.Mult[:] = (
            self.BPult * (7 * COLUMN_WIDTH - 10 * self.FtgLength) ** 2
        ) / 800

    def calc_Astshr(self):
        self.Astshr[:] = (5 * self.Mult) / (2 * self.Dom)

    def calc_Astmin(self):
        self.Astmin[:] = (228 * self.FtgDepth**2 * np.sqrt(self.fc)) / self.Dom

    def calc_Ast(self):
        self.Ast[:] = np.maximum(self.Astmin, self.Astshr)

    def calc_alpha(self):
        self.alpha[:] = 0.85 - 0.0015 * self.fc

    def calc_gamma(self):
        self.gamma[:] = 0.97 - 0.0025 * self.fc

    def calc_ku(self):
        self.ku[:] = self.Ast / (
            2000 * self.alpha * self.Dom * self.fc * self.gamma * self.FtgLength
        )

    def calc_phi(self):
        self.phi[:] = np.minimum(0.85, np.maximum(0.65, 1.24 - 1.08333 * self.ku))

    def calc_Astact(self):
        self.Astact[:] = (250000 * self.ReoSize**2 * np.pi) / self.ReoCts

    # phi Muo per metre
    def calc_fMuo(self):
        self.fMuo[:] = (self.Astact * self.Dom * self.phi) / 2 - (
            self.Astact**2 * self.phi
        ) / (8000 * self.alpha * self.fc * self.FtgLength)

    def calc_M_Ratio(self):
        self.M_Ratio[:] = self.Mult / self.fMuo

    def calc_cost(self):
        self.cost[:] = (
            self.Astact / 1000000 * self.FtgLength * REO_DENSITY * 2 * 3400
            + self.FtgLength** 2
            * self.FtgDepth
            * (130.866 * np.exp(self.fc * 0.0111) + 45 + 130)
            + 4 * self.FtgDepth
        )

    def remove_fails(self):
        self.sizes = self.sizes[
            (self.M_Ratio <= 1)
            & (self.VL_Ratio <= 1)
            & (self.VP_Ratio <= 1)
            & (self.Astact >= self.Ast)
            & (self.BP_Ratio <= 1)
        ]

    def filter_match(
        self, FtgLength=None, FtgDepth=None, fc=None, ReoSize=None, ReoCts=None
    ):
        """Filter the sizes array based on the provided criteria."""
        conditions = []
        if FtgLength is not None:
            conditions.append(self.sizes[:, 0] == FtgLength)
        if FtgDepth is not None:
            conditions.append(self.sizes[:, 1] == FtgDepth)
        if fc is not None:
            conditions.append(self.sizes[:, 2] == fc)
        if ReoSize is not None:
            conditions.append(self.sizes[:, 3] == ReoSize)
        if ReoCts is not None:
            conditions.append(self.sizes[:, 4] == ReoCts)

        if conditions:
            combined_condition = np.logical_and.reduce(conditions)
            self.sizes = self.sizes[combined_condition]

    def sort_by_cost(self):
        """Sort the foundation sizes by cost from lowest to highest."""
        cost_index = 32
        sorted_indices = np.argsort(self.sizes[:, cost_index])
        self.sizes = self.sizes[sorted_indices]

    def print_raw_array(self, num_rows=5):
        """Print the raw array of foundation sizes."""
        print(self.sizes[0, :])

    def create_foundation_sizes_df(self):
        """Create a DataFrame of all foundation sizes."""
        column_headers = [
            attr
            for attr in self.__dict__
            if isinstance(getattr(self, attr), np.ndarray)
        ][1:]
        num_columns = len(column_headers)

        df = pd.DataFrame(
            self.sizes[:, :num_columns],
            columns=column_headers[:num_columns],
        )
        return df

    def print_foundation_sizes(self, num_rows=6):
        """Print a sample of the foundation sizes."""
        if self.sizes.size == 0:
            print("No foundation sizes available.")
            return

        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        df = self.create_foundation_sizes_df()

        total_rows = df.shape[0]
        step = max(1, total_rows // (num_rows - 1))

        selected_rows = np.arange(0, total_rows, step)
        if len(selected_rows) < num_rows:
            selected_rows = np.append(selected_rows, total_rows - 1)
        elif len(selected_rows) > num_rows:
            selected_rows = selected_rows[:num_rows]

        print(df.iloc[selected_rows].to_string(header=True))

    def print_first_foundation_sizes(self, num_rows=6):
        """Print the first X rows of the foundation sizes."""
        if self.sizes.size == 0:
            print("No foundation sizes available.")
            return

        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        df = self.create_foundation_sizes_df()

        print(df.head(num_rows).to_string(header=True))


# Create an instance of the FoundationSizes class
foundation = FoundationSizes()
foundation.remove_fails()
foundation.sort_by_cost()
# foundation.filter_match(ReoSize=0.016, ReoCts=0.3)
foundation.print_first_foundation_sizes()

   FtgLength  FtgDepth     fc  ReoSize  ReoCts   BPmax  BP_Ratio    Pult   Dom   BPult     CLR   VPult     fVP  VP_Ratio    dv   VLult   kvo    kv    ks    fVuc  VL_Ratio   Mult  Astshr  Astmin     Ast  alpha  gamma    ku   phi  Astact   fMuo  M_Ratio      cost
0      1.600     0.400 20.000    0.012   0.200 236.775     0.947 787.373 0.319 307.567 159.000 628.372 924.080     0.680 0.288  95.961 0.146 0.146 0.857 112.454     0.853 66.988 524.986 511.422 524.986  0.820  0.920 0.034 0.850 565.487 75.371    0.889 48643.602
1      1.650     0.400 20.000    0.012   0.200 222.786     0.891 787.841 0.319 289.381 149.599 638.242 924.080     0.691 0.288  97.522 0.146 0.146 0.857 112.454     0.867 67.892 532.073 511.422 532.073  0.820  0.920 0.033 0.850 565.487 75.410    0.900 50174.879
2      1.700     0.400 20.000    0.012   0.200 210.012     0.840 788.323 0.319 272.776 141.015 647.309 924.080     0.700 0.288  98.745 0.146 0.146 0.857 112.454     0.878 68.753 538.818 511.422 538.818  0.820  0.92